In [1]:
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Carga de datos

In [2]:
df = pd.read_csv('../data/corpus.csv')

In [3]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,sentiment,category,processedReview
0,A36BI9SPO3LTC8,B0023ZQDEC,Gerbera Daisy,"[0, 0]",Not a good fit for me. Much too big. I didn't ...,3,Not good for me,1360972800,"02 16, 2013",0,clothing,good fit much enough bust fit back may work so...
1,AL1WTXQJCKT8X,B00HWF7KPY,"Granny ""Joyous-soul""","[0, 0]",good fit- not to thin- love the less thicker ...,5,Surprised,1403308800,"06 21, 2014",0,clothing,good love little thick weist band many read ta...
2,A2QOT4L1QFYYIW,B003VFGW76,lu shenglan,"[0, 0]",Quality is very goodMy husband liked itThe nex...,5,Quality is very good,1387152000,"12 16, 2013",0,clothing,quality goodmy husband like itthe next time bu...
3,A2KQUKZ4X13BI0,B001CGW432,musica_al,"[2, 2]",I like wearing bungee style slip on shoes. Th...,5,Just what I wanted,1305331200,"05 14, 2011",0,clothing,like wear bungee style slip can limit shoe wan...
4,ATJOVP7P6JRPU,B005OBWI3W,T.W. Connell,"[0, 0]",I thought the bracelet would be packaged in a ...,3,Nice product poor packaging,1355356800,"12 13, 2012",0,clothing,think bracelet package simple box arrive packa...


# Conjuntos de train y test

Creamos los conjuntos de entrenamiento (75% del total) y test (25%).

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    df['processedReview'],
    df['sentiment'],
    train_size=0.75,
    test_size=0.25,
    random_state=42,
    shuffle=True,
    stratify=df['sentiment']
)

La extracción de características será realizada con la clase `TfidfVectorizer`de `sklearn`. Los parámetros y valores que se definen son:
- **ngram_range**: (1, 3), de cara a codificar de manera más rica el contexto de cada palabra se trabajará con 1-grams, 2-grams y 3-grams.
- **max_df**: 0.98, al haber realizado una etapa de preprocesado en la que se eliminan stopwords y se lemmatiza no se requiere volver a eliminar tokens muy frecuentes.
- **min_df**: 3, tokens con una frecuencia baja se interpretan como outliers, posiblemente typos u otro tipo de errores.
- **max_features**: 10000, al trabajar con 1-gram, 2-grams y 3-grams seleccionamos un tamaño de vocabulario grande que nos permita contemplar los top n-grams más representativos.

In [5]:
cv = TfidfVectorizer(
    ngram_range=(1, 3),
    max_df=0.98,
    min_df=3,
    max_features=10000
)

In [6]:
cv.fit(X_train)

X_train_ = cv.transform(X_train)
X_test_ = cv.transform(X_test)    

# Modelado

Para este apartado empelaremos dos modelos que son adecuados para tareas de clasificación binaria con variables discretas. La aproximación será simple en el sentido de que no se destinarán esfuerzos al tuning de hiperparametros.

## Regresión Logística

In [7]:
lr_model = LogisticRegression(C=1, solver='lbfgs')

In [8]:
lr_model.fit(X_train_, y_train)

LogisticRegression(C=1)

In [40]:
lr_train_predict = lr_model.predict(X_train_)
lr_test_predict = lr_model.predict(X_test_)
lr_test_predict_proba = lr_model.predict_proba(X_test_)[:, 1]

In [41]:
pickle.dump(lr_model, open('../models/lr_model.pkl', 'wb'))

## Multinomial Naive Bayes

In [34]:
mnb_model = MultinomialNB()

In [35]:
mnb_model.fit(X_train_, y_train)

MultinomialNB()

Calculamos las predicciones para test, train y las probabilidades para test.

In [43]:
mnb_train_predict = mnb_model.predict(X_train_)
mnb_test_predict = mnb_model.predict(X_test_)
mnb_test_predict_proba = mnb_model.predict_proba(X_test_)[:, 1]

In [44]:
pickle.dump(mnb_model, open('../models/mnb_model.pkl', 'wb'))

In [45]:
df_preds = pd.DataFrame({
    'y_actual': y_test,
    'y_pred_test_lr': lr_test_predict,
    'y_pred_proba_test_lr': lr_test_predict_proba,
    'y_pred_test_mnb': mnb_test_predict,
    'y_pred_proba_test_mnb': mnb_test_predict_proba
})

df_preds.to_csv('../results/preds.csv', index=False)

# Resultados

In [16]:
def print_model_metrics(y_test, y_test_predict):
    print('Confussion matrix:\n{}'.format(confusion_matrix(y_test, y_test_predict)))
    print('\nClassification report:\n{}'.format(classification_report(y_test, y_test_predict)))
    print('Accuracy score:{}'.format(accuracy_score(y_test, y_test_predict)))

In [17]:
print_model_metrics(y_test, lr_test_predict)

Confussion matrix:
[[1463  410]
 [ 364 1510]]

Classification report:
              precision    recall  f1-score   support

           0       0.80      0.78      0.79      1873
           1       0.79      0.81      0.80      1874

    accuracy                           0.79      3747
   macro avg       0.79      0.79      0.79      3747
weighted avg       0.79      0.79      0.79      3747

Accuracy score:0.7934347477982386


In [18]:
print_model_metrics(y_test, mnb_test_predict)

Confussion matrix:
[[1465  408]
 [ 392 1482]]

Classification report:
              precision    recall  f1-score   support

           0       0.79      0.78      0.79      1873
           1       0.78      0.79      0.79      1874

    accuracy                           0.79      3747
   macro avg       0.79      0.79      0.79      3747
weighted avg       0.79      0.79      0.79      3747

Accuracy score:0.7864958633573526


# Conclusiones

Tras entrenar dos modelos sencillos y calcular algunas métricas para los conjuntos de test podemos determinar:
- El tiempo de cómputo no es problema para ninguno de los dos modelos.
- Ambos modelos devuelven métricas similares.
- Sin apenas trabajo en perfeccionar esta etapa (tuning de hiperparámetros, cross validation, variación en los parámetros en la etapa de extracción de características...) los resultados permiten afirmar que el problema es abordable.
- El mejor modelo, aunque no por mucho, es la Regresión Logística, ya que todas sus métricas son superiores y su tasa de falsos positivos (muy interesante e importante en este tipo de problemas) es menor.
- Sería interesante analizar el performance de modelos de boosting o basados en árboles.

Por tanto, el modelo seleccionado es la Regresión Logística.